# **RAG**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import logging
# Supprimer les messages WARNING venant de pypdf
logging.getLogger("pypdf").setLevel(logging.ERROR)


import os
import time
import json
import google.generativeai as genai
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA


# 1. Configure l'API Gemini
os.environ["GOOGLE_API_KEY"] = "cle_api"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# 2. Liste des fichiers PDF
pdf_files = [
    "/content/Angles_et_parallelisme.pdf",
    "/content/Calcul_litteral_expressions_simples.pdf",
    "/content/Coordonnees_dans_le_plan.pdf",
    "/content/Multiplication_et_division_des_relatifs.pdf",
    "/content/Proportionnalite_et_echelles.pdf",
    "/content/Equations_du_1er_degre_a_une_inconnue.pdf",
    "/content/Triangles_et_proprietes.pdf",
    "/content/Theoreme_de_Pythagore.pdf",
    "/content/Développement et factorisation d'expressions",
    "/content/Équations du 1er degré",
    "/content/Inéquations simples",
    "/content/Fonctions numériques : représentation graphique",
    "/content/Théorème de Thalès",
    "/content/Théorème de Pythagore (général)",
    "/content/Trigonométrie dans le triangle rectangle",
    "/content/Géométrie dans l’espace : parallèles et perpendiculaires",
    "/content/Transformations (symétrie centrale, translation…)",
    "/content/Statistiques : médiane, étendue, histogrammes",
    "/content/Nombres entiers et décimaux",
    "/content/Opérations sur les nombres",
    "/content/Fractions et nombres rationnels",
    "/content/Proportionnalité",
    "/content/Organisation et gestion de données (statistiques)",
    "/content/Figures géométriques usuelles",
    "/content/Droites, segments et angles",
    "/content/Symétrie axiale",
    "/content/Périmètre et aire des figures planes",
    "/content/Nombres relatifs : addition et soustraction",
    "/content/Multiplication et division des relatifs",
    "/content/Proportionnalité et échelles",
    "/content/Puissances d’un nombre",
    "/content/Calcul littéral : expressions simples",
    "/content/Équations du 1er degré à une inconnue",
    "/content/Angles et parallélisme",
    "/content/Triangles et propriétés (bissectrices, médiatrices…)",
    "/content/Théorème de Pythagore (cas particulier)",
    "/content/Coordonnées dans le plan",
    "/content/Statistiques : moyenne, représentations graphiques",
    "/content/Nombres rationnels et calculs"
]

# 3. Configuration du modèle et des embeddings
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)

# 4. Fichier final de sortie
output_file = "resultats_questions.txt"

# 5. Boucle sur chaque cours
for pdf_path in pdf_files:
    print(f"\n[INFO] Traitement du cours : {pdf_path}")
    total_questions = 0
    cours_name = os.path.basename(pdf_path).replace(".pdf", "")

    # Charger et découper le cours
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.split_documents(documents)
    vectorstore = FAISS.from_documents(chunks, embedding)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )

    # Générer des questions jusqu'à au moins 400
    while total_questions < 500:
        question_prompt = f"""
        À partir du contenu de ce cours ({cours_name}) et vos connaissances, génère 20 questions pratiques
        (avec leurs réponses détaillées) sous forme d'exercices d'application, de préférence progressifs (faciles à plus difficiles),
        qui permettent de bien comprendre les notions abordées.
        Inclus la question, la réponse correcte et une explication concise de la réponse.
        Le format de sortie DOIT être un objet JSON avec les clés suivantes : "cours", "question", "reponse", "explication".
        """

        try:
            result = rag_chain.invoke({"query": question_prompt})
            output_text = result["result"]

            # Écriture dans le fichier texte (append mode)
            with open(output_file, "a", encoding="utf-8") as f:
                f.write(f"\n===== COURS : {cours_name} | BLOC DE {total_questions + 1} à {total_questions + 20} QUESTIONS =====\n")
                f.write(output_text)
                f.write("\n\n")

            print(f"[SUCCÈS] +20 questions générées pour {cours_name}")
            total_questions += 20
            time.sleep(5)  # Petite pause pour éviter surcharge API

        except Exception as e:
            print(f"[ERREUR] {e} — pause 10 sec puis on réessaie")
            time.sleep(10)

print("\nGénération terminée pour tous les cours. Les résultats sont dans : resultats_questions.txt")


# **LLM**

In [ ]:
import google.generativeai as genai
import json
import os
from dotenv import load_dotenv
import time

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Configuration de l'API Gemini
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    print("La clé API Google n'a pas été trouvée dans le fichier .env.")
    exit()
genai.configure(api_key=GOOGLE_API_KEY)

# Utiliser le nom de modèle correct
model_name = 'gemini-2.0-flash'
model = genai.GenerativeModel(model_name)

# Liste des cours de niveau collège
cours_college = [
    "Développement et factorisation d'expressions",
    "Équations du 1er degré",
    "Inéquations simples",
    "Fonctions numériques : représentation graphique",
    "Théorème de Thalès",
    "Théorème de Pythagore (général)",
    "Trigonométrie dans le triangle rectangle",
    "Géométrie dans l’espace : parallèles et perpendiculaires",
    "Transformations (symétrie centrale, translation…)",
    "Statistiques : médiane, étendue, histogrammes",
    "Nombres entiers et décimaux",
    "Opérations sur les nombres",
    "Fractions et nombres rationnels",
    "Proportionnalité",
    "Organisation et gestion de données (statistiques)",
    "Figures géométriques usuelles",
    "Droites, segments et angles",
    "Symétrie axiale",
    "Périmètre et aire des figures planes",
    "Nombres relatifs : addition et soustraction",
    "Multiplication et division des relatifs",
    "Proportionnalité et échelles",
    "Puissances d’un nombre",
    "Calcul littéral : expressions simples",
    "Équations du 1er degré à une inconnue",
    "Angles et parallélisme",
    "Triangles et propriétés (bissectrices, médiatrices…)",
    "Théorème de Pythagore (cas particulier)",
    "Coordonnées dans le plan",
    "Statistiques : moyenne, représentations graphiques",
    "Nombres rationnels et calculs"
]

def generate_prompt(cours):
    return f"""Génère 1 question-réponse de mathématiques de niveau collège pour le cours suivant :
{cours}.

Inclus la question, la réponse correcte et une explication concise de la réponse.
Le format de sortie DOIT être un objet JSON avec les clés suivantes : "cours", "question", "reponse", "explication".
"""

# Nombre total de questions souhaité
nombre_questions_cible = 5000
all_data = []
appels_par_cours = (nombre_questions_cible + len(cours_college) - 1) // len(cours_college)

# Nom du fichier de sauvegarde
nom_fichier_sauvegarde = "dataset_questions_responses_1.json"

# Délai d'attente en secondes entre les requêtes
DELAI_ATTENTE = 5

print(f"Début de la génération de {nombre_questions_cible} questions.")

# Tenter de charger les données existantes si le fichier existe
try:
    with open(nom_fichier_sauvegarde, "r", encoding="utf-8") as f:
        all_data = json.load(f)
    print(f"Données existantes chargées depuis '{nom_fichier_sauvegarde}'. Total de questions chargées : {len(all_data)}")
except FileNotFoundError:
    print(f"Le fichier '{nom_fichier_sauvegarde}' n'existe pas encore. Démarrage avec un nouveau dataset.")
    all_data = []
except json.JSONDecodeError:
    print(f"Erreur lors de la lecture du fichier '{nom_fichier_sauvegarde}'. Démarrage avec un nouveau dataset.")
    all_data = []

for cours in cours_college:
    print(f"\n--- Début du traitement du cours : {cours} ---")
    for i in range(appels_par_cours):
        print(f"  Appel API {i + 1}/{appels_par_cours} pour le cours : {cours}")
        prompt = generate_prompt(cours)
        try:
            response = model.generate_content(prompt)
            response.resolve()
            json_data_str = response.text.strip()
            if json_data_str.startswith("```json"):
                json_data_str = json_data_str[7:]
            if json_data_str.endswith("```"):
                json_data_str = json_data_str[:-3]

            # Try to remove/replace potential control characters
            json_data_str = ''.join(char for char in json_data_str if ord(char) >= 32)

            try:
                data = json.loads(json_data_str)
                if isinstance(data, dict) and all(key in data for key in ["cours", "question", "reponse", "explication"]):
                    all_data.append(data)
                    print(f"    Question stockée. Total de questions stockées : {len(all_data)}")
                    # Sauvegarder immédiatement
                    with open(nom_fichier_sauvegarde, "w", encoding="utf-8") as f:
                        json.dump(all_data, f, ensure_ascii=False, indent=4)
                elif isinstance(data, list) and len(data) == 1 and all(key in data[0] for key in ["cours", "question", "reponse", "explication"]):
                    all_data.append(data[0])
                    print(f"    Question stockée (format liste). Total de questions stockées : {len(all_data)}")
                    # Sauvegarder immédiatement
                    with open(nom_fichier_sauvegarde, "w", encoding="utf-8") as f:
                        json.dump(all_data, f, ensure_ascii=False, indent=4)
                else:
                    print(f"    Erreur de format pour le cours '{cours}': Réponse après nettoyage : {data}")
            except json.JSONDecodeError as e:
                print(f"    Erreur de décodage JSON pour le cours '{cours}' : {e}")
                if 'response' in locals() and hasattr(response, 'text'):
                    print("    Réponse brute de l'API (avant nettoyage) :", response.text)
            time.sleep(DELAI_ATTENTE) # Attendre avant la prochaine requête

        except genai.GenerativeModelError as e:
            if e.status_code == 429:
                print(f"    Erreur de quota atteinte pour le cours '{cours}'. Veuillez attendre avant de relancer.")
                time.sleep(60) # Attendre plus longtemps en cas d'erreur de quota
            else:
                print(f"    Une erreur s'est produite lors de l'appel à l'API pour le cours '{cours}': {e}")
                if 'response' in locals() and hasattr(response, 'text'):
                    print("    Réponse brute de l'API :", response.text)
        except Exception as e:
            print(f"    Une erreur s'est produite lors de la génération pour le cours '{cours}': {e}")
            if 'response' in locals() and hasattr(response, 'text'):
                print("    Réponse brute de l'API :", response.text)

        if len(all_data) >= nombre_questions_cible:
            print(f"\nNombre cible de {nombre_questions_cible} questions atteint. Arrêt de la génération pour ce cours.")
            break
    if len(all_data) >= nombre_questions_cible:
        print(f"\nNombre cible de {nombre_questions_cible} questions atteint. Arrêt de la génération globale.")
        break

print(f"\nTraitement terminé. {len(all_data)} entrées générées et sauvegardées dans '{nom_fichier_sauvegarde}'.")